In [ ]:
# Water Consumption Prediction using Neural Networks (LSTM)

# 1. Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [ ]:

# 2. Load Dataset
# Example: replace with your CSV file (must contain "Date" and "Water_Consumption")
df = pd.read_csv("water_consumption.csv", parse_dates=["Date"], index_col="Date")

# Sort by date
df = df.sort_index()

# 3. Visualize Data
plt.figure(figsize=(12,5))
plt.plot(df.index, df["Water_Consumption"], label="Water Consumption")
plt.xlabel("Date")
plt.ylabel("Consumption")
plt.title("Water Consumption Over Time")
plt.legend()
plt.show()

# 4. Preprocess Data
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(df[["Water_Consumption"]])

# Define sequence length
sequence_length = 30  # using past 30 days to predict next day

X, y = [], []
for i in range(len(scaled_data) - sequence_length):
    X.append(scaled_data[i:i+sequence_length])
    y.append(scaled_data[i+sequence_length])

X, y = np.array(X), np.array(y)

# Train-test split (80% train, 20% test)
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]






In [ ]:
# 5. Build LSTM Model
model = Sequential()
model.add(LSTM(64, return_sequences=True, input_shape=(sequence_length, 1)))
model.add(Dropout(0.2))
model.add(LSTM(64, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(1))

model.compile(optimizer="adam", loss="mse")

# 6. Train Model
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1)

# 7. Evaluate Model
loss = model.evaluate(X_test, y_test)
print("Test Loss (MSE):", loss)


In [ ]:
# 8. Make Predictions
y_pred = model.predict(X_test)
y_pred_rescaled = scaler.inverse_transform(y_pred)
y_test_rescaled = scaler.inverse_transform(y_test)

In [ ]:
# 9. Plot Predictions vs Actual
plt.figure(figsize=(12,5))
plt.plot(y_test_rescaled, label="Actual")
plt.plot(y_pred_rescaled, label="Predicted")
plt.xlabel("Time Step")
plt.ylabel("Water Consumption")
plt.title("Water Consumption Prediction (LSTM)")
plt.legend()
plt.show()